In [ ]:
import requests, os, re, json
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

In [ ]:
# Define function to get dataframe containing past top MVP candidates
def get_df(url, year):
    res = requests.get(url)

    try:
        res.raise_for_status()
    except Exception as exc:
        print('There was a problem: %s' % exc)

    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, features="html.parser")

    table = soup.select("table#mvp")[0]
    useless_header = table.find('tr', class_='over_header')
    useless_header.decompose()
    table_html = str(table)
    table_io = StringIO(table_html)
    df = pd.read_html(table_io)[0]
    df["Year"] = year
    return df

In [ ]:
# Create years list
years = list(range(2022,2024))

# Create folder to contain mvp data
os.makedirs("past_mvps", exist_ok=True)

# Loop through the years and create CSV files to contain data from each year
for year in years:
    url = f"https://www.basketball-reference.com/awards/awards_{year}.html"
    data = get_df(url, year)
    data.to_csv(f"past_mvps/mvps_{year}.csv", index=False)